<a href="https://colab.research.google.com/github/Koyter/GB_recommend/blob/main/HW1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**

In [1]:
import pandas as pd
import numpy as np

### 1. Hit rate

In [2]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [6]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
  return hit_rate(recommended_list[:k], bought_list)

In [8]:
recommended_list = [143,156, 1134, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]

In [5]:
hit_rate(recommended_list, bought_list)

1

In [7]:
hit_rate_at_k(recommended_list, bought_list)

1

### 2. Precision

In [16]:
recommended_list = [1,3,8,6,9,2,5,10] #id товаров
bought_list = [1,5,6]

In [10]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [11]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [12]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [17]:
precision(recommended_list, bought_list)

0.375

In [18]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [30]:
prices = [100, 200, 300, 400, 500, 600, 700, 800]
money_precision_at_k(recommended_list, bought_list, prices)

0.3333333333333333

### 3. Recall

In [32]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
  return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)

    flags = np.isin(recommend_list[:k], bought_list)
    recall = np.dot(flags, prices_recommended[:k]).sum() / prices_bought.sum()
    
    return recall

In [21]:
recall(recommended_list, bought_list)

1.0

In [22]:
recall_at_k(recommended_list,bought_list,k=10)

1.0

In [34]:
prices_bought = [200, 300, 400, 500, 600]
money_recall_at_k(recommended_list, bought_list, prices, prices_bought, k=5)

0.25

# Метрики ранжирования

## AP@k

In [35]:
recommended_list = [22, 21, 4, 1, 234, 232432,234234,666] #id товаров
bought_list = [1, 221, 3, 4,]

In [36]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [37]:
ap_k(recommended_list, bought_list, k=5)

0.41666666666666663

### MAP@k

In [38]:
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [42]:
def map_k(recommended_list, bought_list, k=5):

    result = [ap_k(recommended_list[i], bought_list[i], k) for i in range(len(bought_list))]
    
    return sum(result) / len(result)

In [43]:
map_k(recommended_list_3_users, bought_list_3_users)

0.3333333333333333

### Mean Reciprocal Rank ( MRR@k )

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [57]:
def mrr_rank(recommended_list, bought_list, k=1):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    return 1 / (relevant_indexes[0] + 1)

def reciprocal_rank(recommended_list, bought_list, k=1):
    result = [mrr_rank(recommended_list[i], bought_list[i], k) for i in range(k)]
    result = np.array(result)

    return result.mean()

In [58]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=3)

0.3333333333333333

Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)



1.   Cumulative Gain at K
2.   Ранговый коэффициент корреляции Кендэлла
3.   Ранговый коэффициент корреляции Спирмена
4.   Expected reciprocal rank (ERR)
5.   PFound

